# Vision Edge-AI와 3D Depth Camera

3D Depth Camera는 공간의 깊이 정보를 캡처하여 3차원 데이터로 변환하는 장치입니다. 이러한 카메라는 다양한 응용 분야에서 사용되며, 컴퓨터 비전, 로봇 공학, 증강현실(AR), 가상현실(VR), 의료 이미지 처리 등에서 중요합니다. 본 강의에서는 3D Depth Camera의 원리, 기술, 응용 사례, 그리고 DeepStream을 활용할 수 있는 방법에 대해 설명합니다.

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 3D Depth Camera의 원리
1. **스테레오 비전 (Stereo Vision)**:
    * 두 개의 카메라로 물체를 촬영한 이미지를 비교하여 깊이를 계산합니다.
    * 사람의 시각 시스템과 유사한 방식입니다.

2. **구조광 (Structured Light)**:
    * 특정 패턴의 빛을 물체에 투사하고, 왜곡된 패턴을 분석하여 깊이를 측정합니다.
    * 대표적인 예: Microsoft Kinect

3. **비행시간 (Time of Flight, ToF)**:
    * 빛이 물체에 반사되어 돌아오는 시간을 측정하여 깊이를 계산합니다.
    * 정확하고 빠른 깊이 측정이 가능합니다.

4. **LiDAR (Light Detection and Ranging)**:
    * 레이저를 사용해 주변 환경을 스캔하여 깊이를 측정합니다.
    * 자율주행차에 주로 사용됩니다.

### 응용 분야
1. **산업 및 제조**:
    * 로봇 비전: 로봇의 정확한 동작과 위치 결정을 지원.
    * 품질 검사: 제품의 결함 및 크기 측정.
2. **의료**:
    * 3D 스캔: 인체를 스캔하여 맞춤형 의료 장비 제작.
    * 수술 보조: 정확한 수술 계획을 위한 3D 모델 생성.
3. **엔터테인먼트**:
    * 증강현실/가상현실(AR/VR): 몰입형 환경 구축.
    * 게임: 사용자 동작을 추적하여 인터랙션 제공.
4. **도매(Retail)**:
    * 고객 행동 분석: 매장 내 고객의 동선을 추적하고 행동을 분석.
    * 재고 관리: 제품 크기 및 위치를 자동으로 스캔하여 재고를 효율적으로 관리.
    * 무인 매장: 3D 데이터를 활용해 고객과 제품의 상호작용을 자동화.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

---

# 간단한 예제 실습

## Jetson Orin Nano에서 OAK-D PRO setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 필수 라이브러리 설치
라이브러리 설치
```bash
# 설치되어있는 OpenCV 삭제
$ pip uninstall opencv-python

# 저장소 추가
$ sudo add-apt-repository universe
$ sudo apt update

# 필수 라이브러리 설치
$ sudo apt install -y build-essential cmake git pkg-config libgtk-3-dev libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev libx264-dev libjpeg-dev libpng-dev libtiff-dev gfortran openexr libatlas-base-dev python3-dev python3-numpy libtbb2 libtbb-dev libdc1394-dev

# OpenCV 설치
pip install opencv-python
```

OAK-D 관련 드라이버 설치
```bash
$ wget -qO- https://docs.luxonis.com/install_dependencies.sh | bash
$ pip install depthai --upgrade
```
* 드라이버 설치 이후 반드시 OAK-D 모듈 연결 USB를 뺐다가 다시 꽂기

In [7]:
import cv2
import depthai as dai

# Create pipeline
pipeline = dai.Pipeline()

# Create ColorCamera node
cam = pipeline.create(dai.node.ColorCamera)
cam.setBoardSocket(dai.CameraBoardSocket.CAM_A)  # CAM_A로 수정
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam.setVideoSize(1920, 1080)

# Create XLinkOut node
xout = pipeline.create(dai.node.XLinkOut)
xout.setStreamName("video")

# Connect camera to output
cam.video.link(xout.input)

# Start the pipeline
with dai.Device(pipeline) as device:
    print("Pipeline started")
    video = device.getOutputQueue(name="video", maxSize=4, blocking=False)
    
    while True:
        # Get frame
        frame = video.get().getCvFrame()
        
        # Show frame
        cv2.imshow("OAK-D Video Feed", frame)
        
        # Exit on 'q' key
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()

RuntimeError: Failed to connect to device, error message: X_LINK_DEVICE_ALREADY_IN_USE

## OAK-D와 DeepStream을 결합한 ObjectDetection

### 준비물
* Jetson
* DeepStream 설치
* OAK-D 카메라 모듈

### 실행 준비
라이브러리 설치
```bash
sudo apt update

# C++ 표준 라이브러리
sudo apt install build-essential g++ libstdc++-12-dev

# nlohmann-json 라이브러리 설치
sudo apt install nlohmann-json3-dev
```

OAK-D를 DeepStream 연결을 위한 DepthAI C++ 라이브러리 플러그인 설치
```bash
$ git clone --recurse-submodules https://github.com/luxonis/depthai-core.git
# 서브모듈 초기화 및 업데이트
$ git submodule update --init --recursive
$ cd depthai-core
$ mkdir build
$ cd build
$ cmake ..
$ make -j$(nproc)

echo "export OPENBLAS_CORETYPE=ARMV8" >> ~/.bashrc

# nlohmann json 다운로드
$ cd ../shared/depthai-shared/include
$ mkdir nlohmann
$ cd nlohmann
$ wget https://github.com/nlohmann/json/releases/download/v3.11.2/json.hpp

#tl/optional.hpp 다운로드
$ cd ..
$ mkdir tl
$ cd tl
$ wget https://raw.githubusercontent.com/TartanLlama/optional/master/include/tl/optional.hpp
```

In [3]:
import depthai as dai
import cv2
import numpy as np
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

# GStreamer 파이프라인 생성 함수
def create_gstreamer_pipeline():
    pipeline_str = """
        appsrc name=source ! videoconvert ! video/x-raw,format=BGR ! \
        nvvideoconvert gpu-id=0 ! video/x-raw(memory:NVMM) ! \
        nvinfer config-file-path=./dstest1_pgie_config.txt ! \
        nvdsosd ! video/x-raw(memory:NVMM),format=RGBA ! \
        nvvidconv ! video/x-raw,format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink name=sink
    """
    return Gst.parse_launch(pipeline_str)

# OAK-D 파이프라인 생성 함수
def create_oak_d_pipeline():
    pipeline = dai.Pipeline()
    cam = pipeline.createColorCamera()
    cam.setPreviewSize(1920, 1080)  # 스트림 너비와 높이를 1920x1080으로 설정
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    cam.setInterleaved(False)
    cam.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

    xout = pipeline.createXLinkOut()
    xout.setStreamName("video")
    cam.preview.link(xout.input)

    return pipeline

# GStreamer에서 처리된 프레임 가져오기
def get_frame_from_gstreamer(sink):
    sample = sink.emit("pull-sample")
    if not sample:
        return None

    buffer = sample.get_buffer()
    caps = sample.get_caps()
    width = caps.get_structure(0).get_value("width")
    height = caps.get_structure(0).get_value("height")

    # 버퍼 데이터를 NumPy 배열로 변환
    success, map_info = buffer.map(Gst.MapFlags.READ)
    if not success:
        return None

    frame = np.ndarray(
        (height, width, 3), dtype=np.uint8, buffer=map_info.data
    )
    buffer.unmap(map_info)

    return frame

# GStreamer로 프레임 전달 함수
def push_frame_to_gstreamer(source, frame):
    # OpenCV 이미지를 GStreamer에 전달 가능한 포맷으로 변환
    height, width, channels = frame.shape

    # GStreamer 버퍼 생성
    buf = Gst.Buffer.new_allocate(None, frame.nbytes, None)
    buf.fill(0, frame.tobytes())
    buf.pts = Gst.util_uint64_scale(Gst.CLOCK_TIME_NONE, 1, Gst.SECOND)

    # Caps 설정
    caps = Gst.Caps.from_string(
        f"video/x-raw,format=BGR,width={width},height={height},framerate=30/1"
    )
    source.set_property("caps", caps)

    # GStreamer로 버퍼 전달
    source.emit("push-buffer", buf)

# 메인 실행
def main():
    gst_pipeline = None  # gst_pipeline 초기화
    try:
        # GStreamer 파이프라인 생성 및 실행
        gst_pipeline = create_gstreamer_pipeline()
        source = gst_pipeline.get_by_name("source")
        sink = gst_pipeline.get_by_name("sink")
        gst_pipeline.set_state(Gst.State.PLAYING)

        # OAK-D 파이프라인 생성
        oak_d_pipeline = create_oak_d_pipeline()

        # OAK-D 장치 초기화 및 데이터 스트림 처리
        with dai.Device(oak_d_pipeline) as device:
            queue = device.getOutputQueue(name="video", maxSize=4, blocking=False)

            while True:
                in_frame = queue.get()
                frame = in_frame.getCvFrame()

                # OpenCV로 OAK-D 원본 프레임 확인 (디버깅용)
                cv2.imshow("OAK-D Frame", frame)

                # GStreamer로 프레임 전달
                push_frame_to_gstreamer(source, frame)

                # GStreamer 처리 결과를 가져와 OpenCV로 디스플레이
                gstreamer_frame = get_frame_from_gstreamer(sink)
                if gstreamer_frame is not None:
                    cv2.imshow("Object Detection", gstreamer_frame)

                if cv2.waitKey(1) == ord('q'):
                    break

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # gst_pipeline이 생성된 경우에만 상태를 NULL로 설정
        if gst_pipeline:
            gst_pipeline.set_state(Gst.State.NULL)
        cv2.destroyAllWindows()
        print("Pipeline stopped and cleaned up.")

# 실행
if __name__ == "__main__":
    main()


Setting min object dimensions as 16x16 instead of 1x1 to support VIC compute mode.
Implicit layer support has been deprecated
INFO: [Implicit Engine Info]: layers num: 0



0:04:42.876802696  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer2> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2092> [UID = 1]: deserialized trt engine from :/home/paymentinapp/Desktop/deepstream/DeepStream/강의자료/03_3D_Depth_Camera/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_int8.engine
0:04:42.876911919  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer2> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2195> [UID = 1]: Use deserialized engine model: /home/paymentinapp/Desktop/deepstream/DeepStream/강의자료/03_3D_Depth_Camera/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_int8.engine
0:04:43.019696089  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer_impl.cpp:343:notifyLoadModelStatus:<nvinfer2> [UID 1]: Load new model:./dstest1_pgie_config.txt sucessfully


[2025-01-20 13:15:33.205] [depthai] [warning] Skipping X_LINK_UNBOOTED device with name "1.2.3" ()


/dvs/git/dirty/git-master_linux/nvutils/nvbufsurftransform/nvbufsurftransform.cpp:4814: => NvVicCompose Failed

/dvs/git/dirty/git-master_linux/nvutils/nvbufsurftransform/nvbufsurftransform.cpp:4819: => RGB/BGR Format transformation is not supported by VIC use GPU instead

0:04:47.859624660  4546 0xffff38000da0 ERROR         nvvideoconvert gstnvvideoconvert.c:4255:gst_nvvideoconvert_transform: buffer transform failed


Pipeline stopped and cleaned up.


In [5]:
import depthai as dai
import cv2
import numpy as np
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

# GStreamer 파이프라인 생성 함수
def create_gstreamer_pipeline():
    pipeline_str = """
        appsrc name=source ! videoconvert ! video/x-raw,format=BGR,width=1920,height=1080 ! \
        nvvideoconvert gpu-id=0 ! video/x-raw(memory:NVMM),format=NV12 ! \
        muxer.sink_0 nvstreammux name=muxer width=1920 height=1080 batch-size=1 batched-push-timeout=40000 ! \
        nvinfer config-file-path=./dstest1_pgie_config.txt ! \
        nvdsosd gpu-id=0 ! \
        nvvidconv gpu-id=0 ! video/x-raw,format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink name=sink
    """
    return Gst.parse_launch(pipeline_str)

# OAK-D 파이프라인 생성 함수
def create_oak_d_pipeline():
    pipeline = dai.Pipeline()
    cam = pipeline.createColorCamera()
    cam.setPreviewSize(1920, 1080)  # 스트림 너비와 높이를 1920x1080으로 설정
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    cam.setInterleaved(False)
    cam.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

    xout = pipeline.createXLinkOut()
    xout.setStreamName("video")
    cam.preview.link(xout.input)

    return pipeline

# GStreamer에서 처리된 프레임 가져오기
def get_frame_from_gstreamer(sink):
    sample = sink.emit("pull-sample")
    if not sample:
        return None

    buffer = sample.get_buffer()
    caps = sample.get_caps()
    width = caps.get_structure(0).get_value("width")
    height = caps.get_structure(0).get_value("height")

    # 버퍼 데이터를 NumPy 배열로 변환
    success, map_info = buffer.map(Gst.MapFlags.READ)
    if not success:
        return None

    frame = np.ndarray(
        (height, width, 3), dtype=np.uint8, buffer=map_info.data
    )
    buffer.unmap(map_info)

    return frame

# GStreamer로 프레임 전달 함수
def push_frame_to_gstreamer(source, frame):
    # OpenCV 이미지를 GStreamer에 전달 가능한 포맷으로 변환
    height, width, channels = frame.shape

    # GStreamer 버퍼 생성
    buf = Gst.Buffer.new_allocate(None, frame.nbytes, None)
    buf.fill(0, frame.tobytes())
    buf.pts = Gst.util_uint64_scale(Gst.CLOCK_TIME_NONE, 1, Gst.SECOND)

    # Caps 설정
    caps = Gst.Caps.from_string(
        f"video/x-raw,format=BGR,width={width},height={height},framerate=30/1"
    )
    source.set_property("caps", caps)

    # GStreamer로 버퍼 전달
    source.emit("push-buffer", buf)

# 메인 실행
def main():
    gst_pipeline = None  # gst_pipeline 초기화
    try:
        # GStreamer 파이프라인 생성 및 실행
        gst_pipeline = create_gstreamer_pipeline()
        source = gst_pipeline.get_by_name("source")
        sink = gst_pipeline.get_by_name("sink")
        gst_pipeline.set_state(Gst.State.PLAYING)

        # OAK-D 파이프라인 생성
        oak_d_pipeline = create_oak_d_pipeline()

        # OAK-D 장치 초기화 및 데이터 스트림 처리
        with dai.Device(oak_d_pipeline) as device:
            queue = device.getOutputQueue(name="video", maxSize=4, blocking=False)

            while True:
                in_frame = queue.get()
                frame = in_frame.getCvFrame()

                # OpenCV로 OAK-D 원본 프레임 확인 (디버깅용)
                cv2.imshow("OAK-D Frame", frame)

                # GStreamer로 프레임 전달
                push_frame_to_gstreamer(source, frame)

                # GStreamer 처리 결과를 가져와 OpenCV로 디스플레이
                gstreamer_frame = get_frame_from_gstreamer(sink)
                if gstreamer_frame is not None:
                    cv2.imshow("Object Detection", gstreamer_frame)

                if cv2.waitKey(1) == ord('q'):
                    break

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # gst_pipeline이 생성된 경우에만 상태를 NULL로 설정
        if gst_pipeline:
            gst_pipeline.set_state(Gst.State.NULL)
        cv2.destroyAllWindows()
        print("Pipeline stopped and cleaned up.")

# 실행
if __name__ == "__main__":
    main()


Setting min object dimensions as 16x16 instead of 1x1 to support VIC compute mode.
Implicit layer support has been deprecated
INFO: [Implicit Engine Info]: layers num: 0



0:10:59.908246985  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer4> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2092> [UID = 1]: deserialized trt engine from :/home/paymentinapp/Desktop/deepstream/DeepStream/강의자료/03_3D_Depth_Camera/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_int8.engine
0:10:59.908478930  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer4> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2195> [UID = 1]: Use deserialized engine model: /home/paymentinapp/Desktop/deepstream/DeepStream/강의자료/03_3D_Depth_Camera/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_int8.engine
0:10:59.911835445  4546 0xaaaad8ea23c0 INFO                 nvinfer gstnvinfer_impl.cpp:343:notifyLoadModelStatus:<nvinfer4> [UID 1]: Load new model:./dstest1_pgie_config.txt sucessfully
/dv

nvstreammux: Successfully handled EOS for source_id=0
Pipeline stopped and cleaned up.


In [12]:
import depthai as dai
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

# GStreamer 파이프라인 생성 함수
def create_gstreamer_pipeline():
    pipeline_str = """
        appsrc name=source ! videoconvert ! video/x-raw,format=BGR ! \
        nvvideoconvert gpu-id=0 ! video/x-raw(memory:NVMM) ! \
        nvstreammux name=muxer width=1920 height=1080 batch-size=1 batched-push-timeout=40000 ! \
        nvinfer config-file-path=./dstest1_pgie_config.txt ! \
        nvdsosd gpu-id=0 ! \
        nvvidconv gpu-id=0 ! video/x-raw,format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink name=sink
    """
    return Gst.parse_launch(pipeline_str)

# OAK-D 파이프라인 생성 함수
def create_oak_d_pipeline():
    pipeline = dai.Pipeline()
    cam = pipeline.createColorCamera()
    cam.setPreviewSize(1920, 1080)  # 해상도 동기화
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    cam.setInterleaved(False)
    cam.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

    xout = pipeline.createXLinkOut()
    xout.setStreamName("video")
    cam.preview.link(xout.input)

    return pipeline

# 메타데이터 디버깅 함수
def process_metadata(sample):
    """GStreamer 추론 결과 메타데이터 디버깅"""
    buf = sample.get_buffer()
    print("Metadata buffer received:", buf)

# GStreamer에서 처리된 프레임 가져오기
def get_frame_from_gstreamer(sink):
    sample = sink.emit("pull-sample")
    if not sample:
        return None

    process_metadata(sample)
    return True

# GStreamer로 프레임 전달 함수
def push_frame_to_gstreamer(source, frame):
    height, width, _ = frame.shape
    buf = Gst.Buffer.new_allocate(None, frame.nbytes, None)
    buf.fill(0, frame.tobytes())
    buf.pts = Gst.util_uint64_scale(Gst.CLOCK_TIME_NONE, 1, Gst.SECOND)

    caps = Gst.Caps.from_string(f"video/x-raw,format=BGR,width={width},height={height},framerate=30/1")
    source.set_property("caps", caps)
    source.emit("push-buffer", buf)

# 메인 실행
def main():
    gst_pipeline = None
    try:
        gst_pipeline = create_gstreamer_pipeline()
        source = gst_pipeline.get_by_name("source")
        sink = gst_pipeline.get_by_name("sink")
        gst_pipeline.set_state(Gst.State.PLAYING)

        oak_d_pipeline = create_oak_d_pipeline()
        with dai.Device(oak_d_pipeline) as device:
            queue = device.getOutputQueue(name="video", maxSize=4, blocking=False)

            while True:
                in_frame = queue.get()
                frame = in_frame.getCvFrame()

                push_frame_to_gstreamer(source, frame)
                if get_frame_from_gstreamer(sink):
                    print("Inference completed.")

    except Exception as e:
        print("Error:", e)
    finally:
        if gst_pipeline:
            gst_pipeline.set_state(Gst.State.NULL)
        print("Pipeline stopped.")

if __name__ == "__main__":
    main()


Error: gst_parse_error: could not link nvvideoconvert11 to muxer, muxer can't handle caps video/x-raw(memory:NVMM) (3)
Pipeline stopped.


In [14]:
import depthai as dai
import gi
import numpy as np
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

# GStreamer 파이프라인 생성 함수
def create_gstreamer_pipeline():
    pipeline_str = """
        appsrc name=source ! videoconvert ! video/x-raw,format=BGR,width=1920,height=1080 ! \
        nvvideoconvert gpu-id=0 ! video/x-raw(memory:NVMM),format=NV12,width=1920,height=1080 ! \
        nvstreammux name=muxer batch-size=1 width=1920 height=1080 batched-push-timeout=40000 ! \
        nvinfer config-file-path=./dstest1_pgie_config.txt ! \
        nvdsosd gpu-id=0 ! \
        nvvidconv ! video/x-raw,format=BGR ! videoconvert ! autovideosink
    """
    return Gst.parse_launch(pipeline_str)

# OAK-D 파이프라인 생성 함수
def create_oak_d_pipeline():
    pipeline = dai.Pipeline()
    cam = pipeline.createColorCamera()
    cam.setPreviewSize(1920, 1080)  # 해상도를 1920x1080으로 설정
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    cam.setInterleaved(False)
    cam.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

    xout = pipeline.createXLinkOut()
    xout.setStreamName("video")
    cam.preview.link(xout.input)

    return pipeline

# GStreamer로 프레임 전달 함수
def push_frame_to_gstreamer(source, frame):
    height, width, channels = frame.shape
    buf = Gst.Buffer.new_allocate(None, frame.nbytes, None)
    buf.fill(0, frame.tobytes())
    buf.pts = Gst.util_uint64_scale(Gst.CLOCK_TIME_NONE, 1, Gst.SECOND)

    caps = Gst.Caps.from_string(f"video/x-raw,format=BGR,width={width},height={height},framerate=30/1")
    source.set_property("caps", caps)
    source.emit("push-buffer", buf)

# GStreamer에서 처리된 메타데이터 확인
def process_metadata(sample):
    buffer = sample.get_buffer()
    print("Metadata buffer received:", buffer)

# GStreamer에서 처리된 프레임 가져오기
def get_frame_from_gstreamer(sink):
    sample = sink.emit("pull-sample")
    if not sample:
        return None

    process_metadata(sample)
    return True

# 메인 실행 함수
def main():
    gst_pipeline = None
    try:
        # GStreamer 파이프라인 생성
        print("Creating GStreamer pipeline...")
        gst_pipeline = create_gstreamer_pipeline()
        source = gst_pipeline.get_by_name("source")
        sink = gst_pipeline.get_by_name("sink")
        gst_pipeline.set_state(Gst.State.PLAYING)
        print("GStreamer pipeline set to PLAYING.")

        # OAK-D 파이프라인 생성
        print("Creating OAK-D pipeline...")
        oak_d_pipeline = create_oak_d_pipeline()
        print("OAK-D pipeline created.")

        # OAK-D 장치 연결 및 데이터 처리
        with dai.Device(oak_d_pipeline) as device:
            print("OAK-D device initialized.")
            queue = device.getOutputQueue(name="video", maxSize=4, blocking=False)

            while True:
                print("Reading frame from OAK-D...")
                in_frame = queue.get()
                frame = in_frame.getCvFrame()

                print("Frame received. Pushing to GStreamer...")
                push_frame_to_gstreamer(source, frame)

                if get_frame_from_gstreamer(sink):
                    print("Inference completed.")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        if gst_pipeline:
            gst_pipeline.set_state(Gst.State.NULL)
        print("Pipeline stopped.")

# 실행
if __name__ == "__main__":
    main()


Creating GStreamer pipeline...
Error: gst_parse_error: could not link nvvideoconvert13 to muxer, muxer can't handle caps video/x-raw(memory:NVMM), format=(string)NV12, width=(int)1920, height=(int)1080 (3)
Pipeline stopped.
